# Modelo de regresión para la estimación de energías activa y reactiva

## Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import sklearn.metrics as metrics
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from itertools import product
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

## Cargue de datos

In [2]:
archivos= []
rango = range(1, 31, 1)
for i in rango:
    nombre = 'DATOSCLIENTE' + str(i) + '.csv'
    archivos.append(nombre)

for i in range(len(archivos)):
    url= 'https://raw.githubusercontent.com/Pacheco-Carvajal/GPA-Data-ElectroDunas/main/' + archivos[i]
    temp = pd.read_csv(url)
    temp['idCliente'] = i + 1
    if i == 0:
        clientes = temp.copy()
    else:
        clientes = pd.concat([clientes, temp])
clientes

,Fecha,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC,idCliente
0,2021-01-01 00:00:00,0.357841,0.282788,455.139171,510.561002,1
1,2021-01-01 01:00:00,0.372264,0.431377,469.978787,469.917178,1
2,2021-01-01 02:00:00,1.044687,0.338626,468.721120,546.949147,1
3,2021-01-01 03:00:00,0.566425,0.495791,452.329255,444.122989,1
4,2021-01-01 04:00:00,1.080556,0.472018,513.477596,535.463719,1
...,...,...,...,...,...,...
11410,2022-04-21 10:00:00,0.960105,0.473234,1273.150602,1027.084539,30
11411,2022-04-21 11:00:00,0.624300,0.699936,1063.524968,1205.829819,30
11412,2022-04-21 12:00:00,0.985633,0.123560,1207.284283,1127.893714,30
11413,2022-04-21 13:00:00,0.710436,0.399262,1205.012971,1090.835898,30


## Depuración de datos

In [3]:
clientes['Fecha'] = pd.to_datetime(clientes['Fecha'])
clientes['Dia'] = clientes['Fecha'].dt.day
clientes['Mes'] = clientes['Fecha'].dt.month
clientes['Año'] = clientes['Fecha'].dt.year
clientes['Hora'] = clientes['Fecha'].dt.hour
clientes['Minuto'] = clientes['Fecha'].dt.minute
clientes['Año_Mes'] = clientes['Fecha'].dt.to_period('M')
clientes = clientes[~(clientes['Año_Mes'] == '2023-04')]
clientes.head()

,Fecha,Active_energy,Reactive_energy,Voltaje_FA,Voltaje_FC,idCliente,Dia,Mes,Año,Hora,Minuto,Año_Mes
0,2021-01-01 00:00:00,0.357841,0.282788,455.139171,510.561002,1,1,1,2021,0,0,2021-01
1,2021-01-01 01:00:00,0.372264,0.431377,469.978787,469.917178,1,1,1,2021,1,0,2021-01
2,2021-01-01 02:00:00,1.044687,0.338626,468.721120,546.949147,1,1,1,2021,2,0,2021-01
3,2021-01-01 03:00:00,0.566425,0.495791,452.329255,444.122989,1,1,1,2021,3,0,2021-01
4,2021-01-01 04:00:00,1.080556,0.472018,513.477596,535.463719,1,1,1,2021,4,0,2021-01


## Energía Activa

### Preprocesamiento de datos

In [4]:
np.random.seed(123)

datos_active_energy = clientes[['Active_energy', 'idCliente', 'Dia', 'Mes', 'Año', 'Hora', 'Minuto']]

X_EA = datos_active_energy.drop('Active_energy', axis=1)
y_EA = datos_active_energy['Active_energy']

X_train_EA, X_test_EA, y_train_EA, y_test_EA = train_test_split(X_EA, y_EA, test_size=0.2)

scaler_EA = StandardScaler()
X_train_scaled_EA = scaler_EA.fit_transform(X_train_EA)
X_test_scaled_EA = scaler_EA.transform(X_test_EA)

In [ ]:
lista_RMSE_EA = []

### Redes Neuronales

In [ ]:
np.random.seed(123)
tf.random.set_seed(123)

def mejor_modelo_nn(X_train_scaled, y_train, X_test_scaled, y_test, num_capas_list, num_neuronas_list, funciones_activacion):
    
    best_model = None
    best_params = {}
    best_rmse = float('inf')  # Inicializamos con infinito para encontrar el mínimo
    
    for num_capas in num_capas_list:
        for num_neuronas in num_neuronas_list:
            for fun_activacion in funciones_activacion:
                print('Modelo entrenado con los siguientes parámetros:')
                print(f'{num_capas} capas, {num_neuronas} neuronas por capa y función de activación {fun_activacion}')
                
                # Crear modelo
                model = Sequential()
                model.add(Dense(num_neuronas, input_dim=X_train_scaled.shape[1], activation=fun_activacion))
                for _ in range(num_capas - 1):
                    model.add(Dense(num_neuronas, activation=fun_activacion))
                model.add(Dense(1))
                
                # Compilar modelo
                model.compile(optimizer='adam', loss='mean_squared_error')
                
                # Entrenar modelo
                model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=0)
                
                # Evaluar modelo
                y_pred = model.predict(X_test_scaled)
                rmse = round(np.sqrt(mean_squared_error(y_test, y_pred)), 2)
                
                print(f'RMSE: {rmse}')
                
                # Actualizar el mejor modelo si corresponde
                if rmse < best_rmse:
                    best_model = model
                    best_params = {'num_capas': num_capas, 'num_neuronas': num_neuronas, 'fun_activacion': fun_activacion}
                    best_rmse = rmse
    
    print('Mejores parámetros:')
    print(best_params)
    print('\nMejor RMSE:', best_rmse)
    
    return best_model, best_params

# Definición de hiperparámetros
num_capas_list = [1, 2, 3]
num_neuronas_list = [32, 64, 128]
funciones_activacion = ['relu', 'sigmoid']

# Selección del mejor modelo y parámetros
best_model_EA, best_params_EA = mejor_modelo_nn(
    X_train_scaled_EA, y_train_EA, X_test_scaled_EA, y_test_EA, num_capas_list, num_neuronas_list, funciones_activacion)

#### Evaluación del mejor del modelo

In [6]:
np.random.seed(123)
tf.random.set_seed(123)

# Creación del modelo
model_NN_EA = Sequential()
model_NN_EA.add(Dense(best_params_EA['num_neuronas'],
                      input_dim=X_train_scaled_EA.shape[1],
                      activation=best_params_EA['fun_activacion']))
for _ in range(best_params_EA['num_capas'] - 1):
    model_NN_EA.add(Dense(best_params_EA['num_neuronas'], activation=best_params_EA['fun_activacion']))
model_NN_EA.add(Dense(1))

# Compilación del modelo
model_NN_EA.compile(optimizer='adam', loss='mean_squared_error')

# Entrenamiento del modelo
model_NN_EA.fit(X_train_scaled_EA, y_train_EA, epochs=10, batch_size=32, verbose=0)

# Evaluación del modelo
y_pred_EA = model_NN_EA.predict(X_test_scaled_EA)

# Cálculo del desempeño del modelo utilizando el RMSE
RMSE_NN_EA = round(np.sqrt(mean_squared_error(y_test_EA, y_pred_EA)), 2)
print(f'El RMSE del modelo es de {RMSE_NN_EA}')

# Se guardan los valores del RMSE
lista_RMSE_EA.append(RMSE_NN_EA)

2897/2897 [==============================] - 3s 928us/step
El RMSE del modelo es de 0.74


### Random Forest

#### Parámetro *max_depth*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de max_depth
RF_max_depth_range_EA = range(1, 21)

# Definición de lista para almacenar el RMSE promedio de cada valor de max_depth
RMSE_RF_max_depth_EA = []

# Uso de un 5-fold cross-validation para cada valor de max_depth
for max_depth in RF_max_depth_range_EA:
    reg_EA = RandomForestRegressor(n_jobs = -1,
                                   max_depth = max_depth)
    RMSE_RF_max_depth_EA.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_EA, X_train_scaled_EA, y_train_EA, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de max_depth
plt.plot(RF_max_depth_range_EA, RMSE_RF_max_depth_EA)
plt.xlabel('max_depth')
plt.ylabel('RMSE')

# Parámetro max_depth que mejor calibra el modelo
best_RF_max_depth_EA = RF_max_depth_range_EA[RMSE_RF_max_depth_EA.index(
    min(RMSE_RF_max_depth_EA))]
print(f'El parámetro max_depth que mejor calibra el modelo es {best_RF_max_depth_EA}')

#### Parámetro *n_estimators*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de n_estimators
RF_n_estimators_range_EA = range(10, 510, 50)

# Definición de lista para almacenar el RMSE promedio de cada valor de n_estimators
RMSE_RF_n_estimators_EA = []

# Uso de un 5-fold cross-validation para cada valor de n_estimators
for n_estimators in RF_n_estimators_range_EA:
    reg_EA = RandomForestRegressor(n_jobs = -1,
                                   max_depth = best_RF_max_depth_EA,
                                   n_estimators = n_estimators)
    RMSE_RF_n_estimators_EA.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_EA, X_train_scaled_EA, y_train_EA, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de n_estimators
plt.plot(RF_n_estimators_range_EA, RMSE_RF_n_estimators_EA)
plt.xlabel('n_estimators')
plt.ylabel('RMSE')

# Parámetro n_estimators que mejor calibra el modelo
best_RF_n_estimators_EA = RF_n_estimators_range_EA[RMSE_RF_n_estimators_EA.index(
    min(RMSE_RF_n_estimators_EA))]
print(f'El parámetro n_estimators que mejor calibra el modelo es {best_RF_n_estimators_EA}')

#### Parámetro *min_samples_leaf*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de min_samples_leaf
RF_min_samples_leaf_range_EA = range(1, 21)

# Definición de lista para almacenar el RMSE promedio de cada valor de min_samples_leaf
RMSE_RF_min_samples_leaf_EA = []

# Uso de un 5-fold cross-validation para cada valor de min_samples_leaf
for min_samples_leaf in RF_min_samples_leaf_range_EA:
    reg_EA = RandomForestRegressor(n_jobs = -1,
                                   max_depth = best_RF_max_depth_EA,
                                   n_estimators = best_RF_n_estimators_EA,
                                   min_samples_leaf = min_samples_leaf)
    RMSE_RF_min_samples_leaf_EA.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_EA, X_train_scaled_EA, y_train_EA, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de min_samples_leaf
plt.plot(RF_min_samples_leaf_range_EA, RMSE_RF_min_samples_leaf_EA)
plt.xlabel('min_samples_leaf')
plt.ylabel('MAE')

# Parámetro min_samples_leaf que mejor calibra el modelo
best_RF_min_samples_leaf_EA = RF_min_samples_leaf_range_EA[RMSE_RF_min_samples_leaf_EA.index(
    min(RMSE_RF_min_samples_leaf_EA))]
print(f'El parámetro min_samples_leaf que mejor calibra el modelo es {best_RF_min_samples_leaf_EA}')

#### Parámetro *min_samples_split*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de min_samples_split
RF_min_samples_split_range_EA = range(2, 21)

# Definición de lista para almacenar el RMSE promedio de cada valor de min_samples_split
RMSE_RF_min_samples_split_EA = []

# Uso de un 5-fold cross-validation para cada valor de min_samples_split
for min_samples_split in RF_min_samples_split_range_EA:
    reg_EA = RandomForestRegressor(n_jobs = -1,
                                   max_depth = best_RF_max_depth_EA,
                                   n_estimators = best_RF_n_estimators_EA,
                                   min_samples_leaf = best_RF_min_samples_leaf_EA,
                                   min_samples_split = min_samples_split)
    RMSE_RF_min_samples_split_EA.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_EA, X_train_scaled_EA, y_train_EA, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de min_samples_split
plt.plot(RF_min_samples_split_range_EA, RMSE_RF_min_samples_split_EA)
plt.xlabel('min_samples_split')
plt.ylabel('RMSE')

# Parámetro min_samples_split que mejor calibra el modelo
best_RF_min_samples_split_EA = RF_min_samples_split_range_EA[RMSE_RF_min_samples_split_EA.index(
    min(RMSE_RF_min_samples_split_EA))]
print(f'El parámetro min_samples_split que mejor calibra el modelo es {best_RF_min_samples_split_EA}')

#### Evaluación del mejor del modelo

In [ ]:
np.random.seed(123)

# Implementación y ajuste del modelo de regresión de Random Forest calibrado con el set de train 
RF_reg_EA = RandomForestRegressor(n_jobs = -1,
                                  max_depth = best_RF_max_depth_EA,
                                  n_estimators = best_RF_n_estimators_EA,
                                  min_samples_leaf = best_RF_min_samples_leaf_EA,
                                  min_samples_split = best_RF_min_samples_split_EA).fit(X_train_scaled_EA, y_train_EA)

# Predicción con el set de test
y_pred_EA = RF_reg_EA.predict(X_test_EA)

# Cálculo del desempeño del modelo utilizando el RMSE
RMSE_RF_EA = round(metrics.mean_squared_error(y_pred_EA, y_test_EA, squared = False), 2)
print(f'El RMSE del modelo es de {RMSE_RF_EA}')

# Se guardan los valores del RMSE
lista_RMSE_EA.append(RMSE_RF_EA)

### XGBoost

#### Parámetro *max_depth*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de max_depth
XGB_max_depth_range_EA = range(1, 21)

# Definición de lista para almacenar el RMSE promedio de cada valor de max_depth
RMSE_XGB_max_depth_EA = []

# Uso de un 5-fold cross-validation para cada valor de max_depth
for max_depth in XGB_max_depth_range_EA:
    reg_EA = XGBRegressor(n_jobs = -1,
                          max_depth = max_depth)
    RMSE_XGB_max_depth_EA.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_EA, X_train_scaled_EA, y_train_EA, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de max_depth
plt.plot(XGB_max_depth_range_EA, RMSE_XGB_max_depth_EA)
plt.xlabel('max_depth')
plt.ylabel('RMSE')

# Parámetro max_depth que mejor calibra el modelo
XGB_best_max_depth_EA = XGB_max_depth_range_EA[RMSE_XGB_max_depth_EA.index(
    min(RMSE_XGB_max_depth_EA))]
print(f'El parámetro max_depth que mejor calibra el modelo es {XGB_best_max_depth_EA}')

#### Parámetro *learning_rate*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de learning_rate
XGB_learning_rate_range_EA = np.arange(0.1, 1, 0.1)

# Definición de lista para almacenar el RMSE promedio de cada valor de learning_rate
RMSE_XGB_learning_rate_EA = []

# Uso de un 5-fold cross-validation para cada valor de learning_rate
for learning_rate in XGB_learning_rate_range_EA:
    reg_EA = XGBRegressor(n_jobs = -1,
                          max_depth = XGB_best_max_depth_EA,
                          learning_rate = learning_rate)
    RMSE_XGB_learning_rate_EA.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_EA, X_train_scaled_EA, y_train_EA, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de learning_rate
plt.plot(XGB_learning_rate_range_EA, RMSE_XGB_learning_rate_EA)
plt.xlabel('learning_rate')
plt.ylabel('RMSE')

# Parámetro learning_rate que mejor calibra el modelo
XGB_best_learning_rate_EA = XGB_learning_rate_range_EA[RMSE_XGB_learning_rate_EA.index(
    min(RMSE_XGB_learning_rate_EA))]
print(f'El parámetro learning_rate que mejor calibra el modelo es {XGB_best_learning_rate_EA}')

#### Parámetro *n_estimators*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de n_estimators
XGB_n_estimators_range_EA = range(10, 510, 50)

# Definición de lista para almacenar el RMSE promedio de cada valor de n_estimators
RMSE_XGB_n_estimators_EA = []

# Uso de un 5-fold cross-validation para cada valor de n_estimators
for n_estimators in XGB_n_estimators_range_EA:
    reg_EA = XGBRegressor(n_jobs = -1,
                          max_depth = XGB_best_max_depth_EA,
                          learning_rate = XGB_best_learning_rate_EA,
                          n_estimators = n_estimators)
    RMSE_XGB_n_estimators_EA.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_EA, X_train_scaled_EA, y_train_EA, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de n_estimators
plt.plot(XGB_n_estimators_range_EA, RMSE_XGB_n_estimators_EA)
plt.xlabel('n_estimators')
plt.ylabel('RMSE')

# Parámetro n_estimators que mejor calibra el modelo
XGB_best_n_estimators_EA = XGB_n_estimators_range_EA[RMSE_XGB_n_estimators_EA.index(
    min(RMSE_XGB_n_estimators_EA))]
print(f'El parámetro n_estimators que mejor calibra el modelo es {XGB_best_n_estimators_EA}')

#### Parámeetro *gamma*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de gamma
XGB_gamma_range_EA = range(0, 501, 50)

# Definición de lista para almacenar el RMSE promedio de cada valor de gamma
RMSE_XGB_gamma_EA = []

# Uso de un 5-fold cross-validation para cada valor de gamma
for gamma in XGB_gamma_range_EA:
    reg_EA = XGBRegressor(n_jobs = -1,
                          max_depth = XGB_best_max_depth_EA,
                          learning_rate = XGB_best_learning_rate_EA,
                          n_estimators = XGB_best_n_estimators_EA,
                          gamma = gamma)
    RMSE_XGB_gamma_EA.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_EA, X_train_scaled_EA, y_train_EA, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de gamma
plt.plot(XGB_gamma_range_EA, RMSE_XGB_gamma_EA)
plt.xlabel('gamma')
plt.ylabel('RMSE')

# Parámetro gamma que mejor calibra el modelo
XGB_best_gamma_EA = XGB_gamma_range_EA[RMSE_XGB_gamma_EA.index(
    min(RMSE_XGB_gamma_EA))]
print(f'El parámetro gamma que mejor calibra el modelo es {XGB_best_gamma_EA}')

#### Parámetro *colsample_bytree*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de colsample_bytree
XGB_colsample_bytree_range_EA = np.arange(0.1, 1, 0.1)

# Definición de lista para almacenar el RMSE promedio de cada valor de colsample_bytree
RMSE_XGB_colsample_bytree_EA = []

# Uso de un 5-fold cross-validation para cada valor de colsample_bytree
for colsample_bytree in XGB_colsample_bytree_range_EA:
    reg_EA = XGBRegressor(n_jobs = -1,
                          max_depth = XGB_best_max_depth_EA,
                          learning_rate = XGB_best_learning_rate_EA,
                          n_estimators = XGB_best_n_estimators_EA,
                          gamma = XGB_best_gamma_EA,
                          colsample_bytree = colsample_bytree)
    RMSE_XGB_colsample_bytree_EA.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_EA, X_train_scaled_EA, y_train_EA, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de colsample_bytree
plt.plot(XGB_colsample_bytree_range_EA, RMSE_XGB_colsample_bytree_EA)
plt.xlabel('colsample_bytree')
plt.ylabel('RMSE')

# Parámetro colsample_bytree que mejor calibra el modelo
XGB_best_colsample_bytree_EA = XGB_colsample_bytree_range_EA[RMSE_XGB_colsample_bytree_EA.index(
    min(RMSE_XGB_colsample_bytree_EA))]
print(f'El parámetro colsample_bytree que mejor calibra el modelo es {XGB_best_colsample_bytree_EA}')

#### Evaluación del mejor del modelo

In [ ]:
np.random.seed(123)

# Implementación y ajuste del modelo de regresión de XGBoost calibrado con el set de train 
XGB_reg_EA = XGBRegressor(n_jobs = -1,
                          max_depth = XGB_best_max_depth_EA,
                          learning_rate = XGB_best_learning_rate_EA,
                          n_estimators = XGB_best_n_estimators_EA,
                          gamma = XGB_best_gamma_EA,
                          colsample_bytree = XGB_best_colsample_bytree_EA).fit(X_train_scaled_EA, y_train_EA)

# Predicción con el set de test
y_pred_EA = XGB_reg_EA.predict(X_test_EA)

# Cálculo del desempeño del modelo utilizando el RMSE y el MAE
RMSE_XGB_EA = round(metrics.mean_squared_error(y_pred_EA, y_test_EA, squared = False), 2)
print(f'El RMSE del modelo es de {RMSE_XGB_EA}')

# Se guardan los valores del RMSE
lista_RMSE_EA.append(RMSE_XGB_EA)

### Comparación de modelos para la estimación de energía activa

In [ ]:
df_RMSE_EA = pd.DataFrame(pd.DataFrame({'Modelo': ['Redes Neuronales', 'Random Forest', 'XGBoost',], 'RMSE': lista_RMSE_EA}))
ax_RMSE_EA = df_RMSE_EA.plot.bar(
    x ='Modelo', y = 'RMSE', rot = 0, figsize = (10, 5), legend = False, xlabel = '', title = 'RMSE Enegía Activa')
for container in ax_RMSE_EA.containers:
    ax_RMSE_EA.bar_label(container)

## Energía Reactiva

In [8]:
np.random.seed(123)

datos_reactive_energy = clientes[['Reactive_energy', 'idCliente', 'Dia', 'Mes', 'Año', 'Hora', 'Minuto']]

X_ER = datos_reactive_energy.drop('Reactive_energy', axis=1)
y_ER = datos_reactive_energy['Reactive_energy']

X_train_ER, X_test_ER, y_train_ER, y_test_ER = train_test_split(X_ER, y_ER, test_size=0.2)

scaler_ER = StandardScaler()
X_train_scaled_ER = scaler_ER.fit_transform(X_train_ER)
X_test_scaled_ER = scaler_ER.transform(X_test_ER)

In [ ]:
lista_RMSE_ER = []

### Redes Neuronales

In [ ]:
np.random.seed(123)

def mejor_modelo_nn(X_train_scaled, y_train, X_test_scaled, y_test, num_capas_list, num_neuronas_list, funciones_activacion):
    
    best_model = None
    best_params = {}
    best_rmse = float('inf')  # Inicializamos con infinito para encontrar el mínimo
    
    for num_capas in num_capas_list:
        for num_neuronas in num_neuronas_list:
            for fun_activacion in funciones_activacion:
                print('Modelo entrenado con los siguientes parámetros:')
                print(f'{num_capas} capas, {num_neuronas} neuronas por capa y función de activación {fun_activacion}')
                
                # Crear modelo
                model = Sequential()
                model.add(Dense(num_neuronas, input_dim=X_train_scaled.shape[1], activation=fun_activacion))
                for _ in range(num_capas - 1):
                    model.add(Dense(num_neuronas, activation=fun_activacion))
                model.add(Dense(1))
                
                # Compilar modelo
                model.compile(optimizer='adam', loss='mean_squared_error')
                
                # Entrenar modelo
                model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=0)
                
                # Evaluar modelo
                y_pred = model.predict(X_test_scaled)
                rmse = round(np.sqrt(mean_squared_error(y_test, y_pred)), 2)
                
                print(f'RMSE: {rmse}')
                
                # Actualizar el mejor modelo si corresponde
                if rmse < best_rmse:
                    best_model = model
                    best_params = {'num_capas': num_capas, 'num_neuronas': num_neuronas, 'fun_activacion': fun_activacion}
                    best_rmse = rmse
    
    print('Mejores parámetros:')
    print(best_params)
    print('\nMejor RMSE:', best_rmse)
    
    return best_model, best_params

# Definición de hiperparámetros
num_capas_list = [1, 2, 3]
num_neuronas_list = [32, 64, 128]
funciones_activacion = ['relu', 'sigmoid']

# Selección del mejor modelo y parámetros
best_model_ER, best_params_ER = mejor_modelo_nn(
    X_train_scaled_ER, y_train_ER, X_test_scaled_ER, y_test_ER, num_capas_list, num_neuronas_list, funciones_activacion)

#### Evaluación del mejor del modelo

In [9]:
np.random.seed(123)

# Creación del modelo
model_NN_ER = Sequential()
model_NN_ER.add(Dense(128,
                      input_dim=X_train_scaled_ER.shape[1],
                      activation='relu'))
for _ in range(3 - 1):
    model_NN_ER.add(Dense(128, activation='relu'))
    model_NN_ER.add(Dense(1))
                
# Compilación del modelo
model_NN_ER.compile(optimizer='adam', loss='mean_squared_error')
                
# Entrenamiento del modelo
model_NN_ER.fit(X_train_scaled_ER, y_train_ER, epochs=10, batch_size=32, verbose=0)
                
# Evaluación del modelo
y_pred_ER = model_NN_ER.predict(X_test_scaled_ER)

# Cálculo del desempeño del modelo utilizando el RMSE
RMSE_NN_ER = round(np.sqrt(mean_squared_error(y_test_ER, y_pred_ER)), 2)
print(f'El RMSE del modelo es de {RMSE_NN_ER}')

# Se guardan los valores del RMSE
# lista_RMSE_ER.append(RMSE_NN_ER)

2897/2897 [==============================] - 3s 986us/step
El RMSE del modelo es de 0.67


### Random Forest

#### Parámetro *max_depth*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de max_depth
RF_max_depth_range_ER = range(1, 21)

# Definición de lista para almacenar el RMSE promedio de cada valor de max_depth
RMSE_RF_max_depth_ER = []

# Uso de un 5-fold cross-validation para cada valor de max_depth
for max_depth in RF_max_depth_range_ER:
    reg_ER = RandomForestRegressor(n_jobs = -1,
                                   max_depth = max_depth)
    RMSE_RF_max_depth_ER.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_ER, X_train_scaled_ER, y_train_ER, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de max_depth
plt.plot(RF_max_depth_range_ER, RMSE_RF_max_depth_ER)
plt.xlabel('max_depth')
plt.ylabel('RMSE')

# Parámetro max_depth que mejor calibra el modelo
best_RF_max_depth_ER = RF_max_depth_range_ER[RMSE_RF_max_depth_ER.index(
    min(RMSE_RF_max_depth_ER))]
print(f'El parámetro max_depth que mejor calibra el modelo es {best_RF_max_depth_ER}')

#### Parámetro *n_estimators*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de n_estimators
RF_n_estimators_range_ER = range(10, 510, 50)

# Definición de lista para almacenar el RMSE promedio de cada valor de n_estimators
RMSE_RF_n_estimators_ER = []

# Uso de un 5-fold cross-validation para cada valor de n_estimators
for n_estimators in RF_n_estimators_range_ER:
    reg_ER = RandomForestRegressor(n_jobs = -1,
                                   max_depth = best_RF_max_depth_ER,
                                   n_estimators = n_estimators)
    RMSE_RF_n_estimators_ER.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_ER, X_train_scaled_ER, y_train_ER, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de n_estimators
plt.plot(RF_n_estimators_range_ER, RMSE_RF_n_estimators_ER)
plt.xlabel('n_estimators')
plt.ylabel('RMSE')

# Parámetro n_estimators que mejor calibra el modelo
best_RF_n_estimators_ER = RF_n_estimators_range_ER[RMSE_RF_n_estimators_ER.index(
    min(RMSE_RF_n_estimators_ER))]
print(f'El parámetro n_estimators que mejor calibra el modelo es {best_RF_n_estimators_ER}')

#### Parámetro *min_samples_leaf*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de min_samples_leaf
RF_min_samples_leaf_range_ER = range(1, 21)

# Definición de lista para almacenar el RMSE promedio de cada valor de min_samples_leaf
RMSE_RF_min_samples_leaf_ER = []

# Uso de un 5-fold cross-validation para cada valor de min_samples_leaf
for min_samples_leaf in RF_min_samples_leaf_range_ER:
    reg_ER = RandomForestRegressor(n_jobs = -1,
                                   max_depth = best_RF_max_depth_ER,
                                   n_estimators = best_RF_n_estimators_ER,
                                   min_samples_leaf = min_samples_leaf)
    RMSE_RF_min_samples_leaf_ER.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_ER, X_train_scaled_ER, y_train_ER, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de min_samples_leaf
plt.plot(RF_min_samples_leaf_range_ER, RMSE_RF_min_samples_leaf_ER)
plt.xlabel('min_samples_leaf')
plt.ylabel('MAE')

# Parámetro min_samples_leaf que mejor calibra el modelo
best_RF_min_samples_leaf_ER = RF_min_samples_leaf_range_ER[RMSE_RF_min_samples_leaf_ER.index(
    min(RMSE_RF_min_samples_leaf_ER))]
print(f'El parámetro min_samples_leaf que mejor calibra el modelo es {best_RF_min_samples_leaf_ER}')

#### Parámetro *min_samples_split*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de min_samples_split
RF_min_samples_split_range_ER = range(2, 21)

# Definición de lista para almacenar el RMSE promedio de cada valor de min_samples_split
RMSE_RF_min_samples_split_ER = []

# Uso de un 5-fold cross-validation para cada valor de min_samples_split
for min_samples_split in RF_min_samples_split_range_ER:
    reg_ER = RandomForestRegressor(n_jobs = -1,
                                   max_depth = best_RF_max_depth_ER,
                                   n_estimators = best_RF_n_estimators_ER,
                                   min_samples_leaf = best_RF_min_samples_leaf_ER,
                                   min_samples_split = min_samples_split)
    RMSE_RF_min_samples_split_ER.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_ER, X_train_scaled_ER, y_train_ER, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de min_samples_split
plt.plot(RF_min_samples_split_range_ER, RMSE_RF_min_samples_split_ER)
plt.xlabel('min_samples_split')
plt.ylabel('RMSE')

# Parámetro min_samples_split que mejor calibra el modelo
best_RF_min_samples_split_ER = RF_min_samples_split_range_ER[RMSE_RF_min_samples_split_ER.index(
    min(RMSE_RF_min_samples_split_ER))]
print(f'El parámetro min_samples_split que mejor calibra el modelo es {best_RF_min_samples_split_ER}')

#### Evaluación del mejor del modelo

In [ ]:
np.random.seed(123)

# Implementación y ajuste del modelo de regresión de Random Forest calibrado con el set de train 
RF_reg_ER = RandomForestRegressor(n_jobs = -1,
                                  max_depth = best_RF_max_depth_ER,
                                  n_estimators = best_RF_n_estimators_ER,
                                  min_samples_leaf = best_RF_min_samples_leaf_ER,
                                  min_samples_split = best_RF_min_samples_split_ER).fit(X_train_scaled_ER, y_train_ER)

# Predicción con el set de test
y_pred_ER = RF_reg_ER.predict(X_test_ER)

# Cálculo del desempeño del modelo utilizando el RMSE
RMSE_RF_ER = round(metrics.mean_squared_error(y_pred_ER, y_test_ER, squared = False), 2)
print(f'El RMSE del modelo es de {RMSE_RF_ER}')

# Se guardan los valores del RMSE
lista_RMSE_ER.append(RMSE_RF_ER)

### XGBoost

#### Parámetro *max_depth*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de max_depth
XGB_max_depth_range_ER = range(1, 21)

# Definición de lista para almacenar el RMSE promedio de cada valor de max_depth
RMSE_XGB_max_depth_ER = []

# Uso de un 5-fold cross-validation para cada valor de max_depth
for max_depth in XGB_max_depth_range_ER:
    reg_ER = XGBRegressor(n_jobs = -1,
                          max_depth = max_depth)
    RMSE_XGB_max_depth_ER.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_ER, X_train_scaled_ER, y_train_ER, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de max_depth
plt.plot(XGB_max_depth_range_ER, RMSE_XGB_max_depth_ER)
plt.xlabel('max_depth')
plt.ylabel('RMSE')

# Parámetro max_depth que mejor calibra el modelo
XGB_best_max_depth_ER = XGB_max_depth_range_ER[RMSE_XGB_max_depth_ER.index(
    min(RMSE_XGB_max_depth_ER))]
print(f'El parámetro max_depth que mejor calibra el modelo es {XGB_best_max_depth_ER}')

#### Parámetro *learning_rate*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de learning_rate
XGB_learning_rate_range_ER = np.arange(0.1, 1, 0.1)

# Definición de lista para almacenar el RMSE promedio de cada valor de learning_rate
RMSE_XGB_learning_rate_ER = []

# Uso de un 5-fold cross-validation para cada valor de learning_rate
for learning_rate in XGB_learning_rate_range_ER:
    reg_ER = XGBRegressor(n_jobs = -1,
                          max_depth = XGB_best_max_depth_ER,
                          learning_rate = learning_rate)
    RMSE_XGB_learning_rate_ER.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_ER, X_train_scaled_ER, y_train_ER, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de learning_rate
plt.plot(XGB_learning_rate_range_ER, RMSE_XGB_learning_rate_ER)
plt.xlabel('learning_rate')
plt.ylabel('RMSE')

# Parámetro learning_rate que mejor calibra el modelo
XGB_best_learning_rate_ER = XGB_learning_rate_range_ER[RMSE_XGB_learning_rate_ER.index(
    min(RMSE_XGB_learning_rate_ER))]
print(f'El parámetro learning_rate que mejor calibra el modelo es {XGB_best_learning_rate_ER}')

#### Parámetro *n_estimators*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de n_estimators
XGB_n_estimators_range_ER = range(10, 510, 50)

# Definición de lista para almacenar el RMSE promedio de cada valor de n_estimators
RMSE_XGB_n_estimators_ER = []

# Uso de un 5-fold cross-validation para cada valor de n_estimators
for n_estimators in XGB_n_estimators_range_ER:
    reg_ER = XGBRegressor(n_jobs = -1,
                          max_depth = XGB_best_max_depth_ER,
                          learning_rate = XGB_best_learning_rate_ER,
                          n_estimators = n_estimators)
    RMSE_XGB_n_estimators_ER.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_ER, X_train_scaled_ER, y_train_ER, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de n_estimators
plt.plot(XGB_n_estimators_range_ER, RMSE_XGB_n_estimators_ER)
plt.xlabel('n_estimators')
plt.ylabel('RMSE')

# Parámetro n_estimators que mejor calibra el modelo
XGB_best_n_estimators_ER = XGB_n_estimators_range_ER[RMSE_XGB_n_estimators_ER.index(
    min(RMSE_XGB_n_estimators_ER))]
print(f'El parámetro n_estimators que mejor calibra el modelo es {XGB_best_n_estimators_ER}')

#### Parámetro *gamma*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de gamma
XGB_gamma_range_ER = range(0, 501, 50)

# Definición de lista para almacenar el RMSE promedio de cada valor de gamma
RMSE_XGB_gamma_ER = []

# Uso de un 5-fold cross-validation para cada valor de gamma
for gamma in XGB_gamma_range_ER:
    reg_ER = XGBRegressor(n_jobs = -1,
                          max_depth = XGB_best_max_depth_ER,
                          learning_rate = XGB_best_learning_rate_ER,
                          n_estimators = XGB_best_n_estimators_ER,
                          gamma = gamma)
    RMSE_XGB_gamma_ER.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_ER, X_train_scaled_ER, y_train_ER, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de gamma
plt.plot(XGB_gamma_range_ER, RMSE_XGB_gamma_ER)
plt.xlabel('gamma')
plt.ylabel('RMSE')

# Parámetro gamma que mejor calibra el modelo
XGB_best_gamma_ER = XGB_gamma_range_ER[RMSE_XGB_gamma_ER.index(
    min(RMSE_XGB_gamma_ER))]
print(f'El parámetro gamma que mejor calibra el modelo es {XGB_best_gamma_ER}')

#### Parámetro *colsample_bytree*

In [ ]:
np.random.seed(123)

# Creación de lista para iterar sobre diferentes valores de colsample_bytree
XGB_colsample_bytree_range_ER = np.arange(0.1, 1, 0.1)

# Definición de lista para almacenar el RMSE promedio de cada valor de colsample_bytree
RMSE_XGB_colsample_bytree_ER = []

# Uso de un 5-fold cross-validation para cada valor de colsample_bytree
for colsample_bytree in XGB_colsample_bytree_range_ER:
    reg_ER = XGBRegressor(n_jobs = -1,
                          max_depth = XGB_best_max_depth_ER,
                          learning_rate = XGB_best_learning_rate_ER,
                          n_estimators = XGB_best_n_estimators_ER,
                          gamma = XGB_best_gamma_ER,
                          colsample_bytree = colsample_bytree)
    RMSE_XGB_colsample_bytree_ER.append(np.mean(np.sqrt(np.abs(
        cross_val_score(reg_ER, X_train_scaled_ER, y_train_ER, cv = 5, scoring = 'neg_mean_squared_error')))))

# Gráfica del desempeño del modelo para cada valor definido de colsample_bytree
plt.plot(XGB_colsample_bytree_range_ER, RMSE_XGB_colsample_bytree_ER)
plt.xlabel('colsample_bytree')
plt.ylabel('RMSE')

# Parámetro colsample_bytree que mejor calibra el modelo
XGB_best_colsample_bytree_ER = XGB_colsample_bytree_range_ER[RMSE_XGB_colsample_bytree_ER.index(
    min(RMSE_XGB_colsample_bytree_ER))]
print(f'El parámetro colsample_bytree que mejor calibra el modelo es {XGB_best_colsample_bytree_ER}')

#### Evaluación del mejor del modelo

In [ ]:
np.random.seed(123)

# Implementación y ajuste del modelo de regresión de XGBoost calibrado con el set de train 
XGB_reg_ER = XGBRegressor(n_jobs = -1,
                          max_depth = XGB_best_max_depth_ER,
                          learning_rate = XGB_best_learning_rate_ER,
                          n_estimators = XGB_best_n_estimators_ER,
                          gamma = XGB_best_gamma_ER,
                          colsample_bytree = XGB_best_colsample_bytree_ER).fit(X_train_scaled_ER, y_train_ER)

# Predicción con el set de test
y_pred_ER = XGB_reg_ER.predict(X_test_ER)

# Cálculo del desempeño del modelo utilizando el RMSE y el MAE
RMSE_XGB_ER = round(metrics.mean_squared_error(y_pred_ER, y_test_ER, squared = False), 2)
print(f'El RMSE del modelo es de {RMSE_XGB_ER}')

# Se guardan los valores del RMSE
lista_RMSE_ER.append(RMSE_XGB_ER)

### Comparación de modelos para la estimación de energía reactiva

In [ ]:
df_RMSE_ER = pd.DataFrame(pd.DataFrame({'Modelo': ['Redes Neuronales', 'Random Forest', 'XGBoost',], 'RMSE': lista_RMSE_ER}))
ax_RMSE_ER = df_RMSE_ER.plot.bar(
    x ='Modelo', y = 'RMSE', rot = 0, figsize = (10, 5), legend = False, xlabel = '', title = 'RMSE Enegía Activa')
for container in ax_RMSE_ER.containers:
    ax_RMSE_ER.bar_label(container)

## Estimación de las demandas de energías activa y reactiva a partir del modelo de redes neuronales

In [163]:
# Creación de un DataFrame con todas las combinaciones posibles de las categorías de las variables predictoras
df1 = pd.DataFrame(
    list(product(datos_active_energy['idCliente'].unique(),
                 pd.date_range(clientes['Fecha'].max() + pd.Timedelta(hours=1), '2024-12-31 23:00:00', freq='H'))),
    columns=['idCliente', 'Fecha'])
df1['Dia'] = df1['Fecha'].dt.day
df1['Mes'] = df1['Fecha'].dt.month
df1['Año'] = df1['Fecha'].dt.year
df1['Hora'] = df1['Fecha'].dt.hour
df1['Minuto'] = df1['Fecha'].dt.minute

# Normalización de las variables predictoras
scaler = StandardScaler()
df1_scaled = scaler.fit_transform(df1.drop('Fecha', axis=1))

# Predicción de las energías activa y reactiva para cada combinación
y_pred_EA = model_NN_EA.predict(df1_scaled)
y_pred_ER = model_NN_ER.predict(df1_scaled)

# Se adición las predicciones al DataFrame de combinaciones
df1['Forecast_Energia_Activa'] = y_pred_EA
df1['Forecast_Energia_Reactiva'] = y_pred_ER

# Transformación de las variables predictoras a su escala original
df1_inverse = scaler.inverse_transform(df1_scaled)
df1[['idCliente', 'Dia', 'Mes', 'Año', 'Hora', 'Minuto']] = df1_inverse

# Se muestra el DataFrame con las predicciones
df1

14423/14423 [==============================] - 16s 1ms/step


,idCliente,Fecha,Dia,Mes,Año,Hora,Minuto,Forecast_Energia_Activa,Forecast_Energia_Reactiva
0,1.0,2023-04-01 00:00:00,1.0,4.0,2023.0,0.0,0.0,2.835779,1.778564
1,1.0,2023-04-01 01:00:00,1.0,4.0,2023.0,1.0,0.0,2.836243,1.765075
2,1.0,2023-04-01 02:00:00,1.0,4.0,2023.0,2.0,0.0,2.836150,1.753164
3,1.0,2023-04-01 03:00:00,1.0,4.0,2023.0,3.0,0.0,2.834813,1.755167
4,1.0,2023-04-01 04:00:00,1.0,4.0,2023.0,4.0,0.0,2.833479,1.757171
...,...,...,...,...,...,...,...,...,...
461515,30.0,2024-12-31 19:00:00,31.0,12.0,2024.0,19.0,0.0,-0.178790,0.190231
461516,30.0,2024-12-31 20:00:00,31.0,12.0,2024.0,20.0,0.0,-0.200064,0.193405
461517,30.0,2024-12-31 21:00:00,31.0,12.0,2024.0,21.0,0.0,-0.220772,0.195346
461518,30.0,2024-12-31 22:00:00,31.0,12.0,2024.0,22.0,0.0,0.544038,0.196260


In [164]:
df2 = pd.DataFrame(
    list(product(clientes['idCliente'].unique(), pd.date_range(clientes['Fecha'].min(), clientes['Fecha'].max(), freq='H'))),
    columns=['idCliente', 'Fecha'])
df2['Dia'] = df2['Fecha'].dt.day
df2['Mes'] = df2['Fecha'].dt.month
df2['Año'] = df2['Fecha'].dt.year
df2['Hora'] = df2['Fecha'].dt.hour
df2['Minuto'] = df2['Fecha'].dt.minute

df2 = df2.merge(clientes[['Active_energy', 'Reactive_energy', 'idCliente', 'Dia', 'Mes', 'Año', 'Hora', 'Minuto']],
                on=['idCliente', 'Dia', 'Mes', 'Año', 'Hora', 'Minuto'],
                how='left')
df2.fillna(0, inplace=True)
df2

,idCliente,Fecha,Dia,Mes,Año,Hora,Minuto,Active_energy,Reactive_energy
0,1,2021-01-01 00:00:00,1,1,2021,0,0,0.357841,0.282788
1,1,2021-01-01 01:00:00,1,1,2021,1,0,0.372264,0.431377
2,1,2021-01-01 02:00:00,1,1,2021,2,0,1.044687,0.338626
3,1,2021-01-01 03:00:00,1,1,2021,3,0,0.566425,0.495791
4,1,2021-01-01 04:00:00,1,1,2021,4,0,1.080556,0.472018
...,...,...,...,...,...,...,...,...,...
590395,30,2023-03-31 19:00:00,31,3,2023,19,0,0.000000,0.000000
590396,30,2023-03-31 20:00:00,31,3,2023,20,0,0.000000,0.000000
590397,30,2023-03-31 21:00:00,31,3,2023,21,0,0.000000,0.000000
590398,30,2023-03-31 22:00:00,31,3,2023,22,0,0.000000,0.000000


In [183]:
# Concatenación los dataframmes df2 y df1
df = pd.concat([df2, df1])

# Transformación de la columna Fecha a tipo datetime
df['Fecha'] = pd.to_datetime(df['Fecha'].dt.strftime('%b-%Y'), errors='coerce')
df = df[['idCliente', 'Fecha', 'Active_energy', 'Reactive_energy', 'Forecast_Energia_Activa', 'Forecast_Energia_Reactiva']]
df = df.groupby(['idCliente', 'Fecha']).sum().reset_index()
df['Año'] = df['Fecha'].dt.year
df

,idCliente,Fecha,Active_energy,Reactive_energy,Forecast_Energia_Activa,Forecast_Energia_Reactiva,Año
0,1.0,2021-01-01,1179.028493,415.713658,0.000000,0.000000,2021
1,1.0,2021-02-01,1490.128945,231.737781,0.000000,0.000000,2021
2,1.0,2021-03-01,1728.240344,978.845693,0.000000,0.000000,2021
3,1.0,2021-04-01,2228.447578,1304.500199,0.000000,0.000000,2021
4,1.0,2021-05-01,2190.919624,1269.936238,0.000000,0.000000,2021
...,...,...,...,...,...,...,...
1435,30.0,2024-08-01,0.000000,0.000000,3838.345947,37.353306,2024
1436,30.0,2024-09-01,0.000000,0.000000,3995.521729,102.005341,2024
1437,30.0,2024-10-01,0.000000,0.000000,3613.117920,468.038483,2024
1438,30.0,2024-11-01,0.000000,0.000000,2782.801758,522.344116,2024


In [176]:
df.to_csv('Regresión EA - ER.csv')

In [165]:
# Concatenación los dataframmes df2 y df1
df = pd.concat([df2, df1])

# Transformación de la columna Fecha a tipo datetime
df['Fecha'] = pd.to_datetime(df['Fecha'].dt.strftime('%b-%Y'), errors='coerce')

# Selección de las columnas necesarias
df = df[['Fecha', 'Active_energy', 'Reactive_energy', 'Forecast_Energia_Activa', 'Forecast_Energia_Reactiva']]

# Agrupación por la columna Fecha sumando los valores de las demás columnas
df = df.groupby(['Fecha']).sum().reset_index()
df

,Fecha,Active_energy,Reactive_energy,Forecast_Energia_Activa,Forecast_Energia_Reactiva
0,2021-01-01,23391.109853,16032.867024,0.000000,0.000000
1,2021-02-01,22288.437982,14672.832897,0.000000,0.000000
2,2021-03-01,26048.402416,18326.167337,0.000000,0.000000
3,2021-04-01,25853.115672,17784.814414,0.000000,0.000000
4,2021-05-01,26534.942537,18724.404675,0.000000,0.000000
5,2021-06-01,25305.670241,17149.131039,0.000000,0.000000
6,2021-07-01,23536.845564,14489.421053,0.000000,0.000000
7,2021-08-01,28949.809435,17226.163106,0.000000,0.000000
8,2021-09-01,30152.117139,18492.604029,0.000000,0.000000
9,2021-10-01,35405.449279,20185.231634,0.000000,0.000000


In [178]:
df.idCliente.unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30.,  5.])

In [169]:
fig = px.line(df.replace(0, ''), x='Fecha', y=['Active_energy', 'Forecast_Energia_Activa'],
              title='Energía Activa vs. Pronóstico de Energía Activa por Mes',
              labels={'value': 'Energía Activa', 'Mes': 'Mes'})
fig.show()

In [170]:
fig = px.line(df.replace(0, ''), x='Fecha', y=['Reactive_energy', 'Forecast_Energia_Reactiva'],
              title='Energía Reactiva vs. Pronóstico de Energía Reactiva por Mes',
              labels={'value': 'Energía Reactiva', 'Mes': 'Mes'})
fig.show()